In [3]:
# Install required packages
import subprocess
import sys

def install_packages():
    packages = [
        'opencv-python',
        'mediapipe',
        'pyautogui',
        'numpy'
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ Successfully installed {package}")
        except subprocess.CalledProcessError:
            print(f"✗ Failed to install {package}")

# Run installation
install_packages()
print("\nAll packages installation completed!")

✓ Successfully installed opencv-python
  Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached jax-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.1-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached sounddevice-0.5.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl (35.6 MB)
Using cached sounddevice-0.5.2-py3-none-any.whl (32 kB)
Using cached j

ERROR: Exception:
Traceback (most recent call last):
  File "/home/tazmeen/anaconda3/lib/python3.12/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/home/tazmeen/anaconda3/lib/python3.12/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/home/tazmeen/anaconda3/lib/python3.12/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/home/tazmeen/anaconda3/lib/python3.12/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "/home/tazmeen/anaconda3/lib/python3.12/http/client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "/home/tazmeen/anaconda3/lib/python3.12/socket.py", line 708,

✗ Failed to install mediapipe
✓ Successfully installed pyautogui
✓ Successfully installed numpy

All packages installation completed!


In [4]:
# Eye-Tracking Virtual Keyboard and Mouse Control System
# This system uses MediaPipe for eye tracking and OpenCV for webcam input

import cv2
import mediapipe as mp
import numpy as np
import pyautogui
import tkinter as tk
from tkinter import ttk
import threading
import time
import math
from collections import deque

# Disable pyautogui failsafe for smooth operation
pyautogui.FAILSAFE = False

class EyeTracker:
    def __init__(self):
        # Initialize MediaPipe face mesh
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Eye landmarks indices
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
        
        # Calibration points for screen mapping
        self.calibration_points = []
        self.screen_width, self.screen_height = pyautogui.size()
        
        # Smoothing buffer for eye positions
        self.eye_positions = deque(maxlen=10)
        
        # Blink detection
        self.blink_threshold = 0.25
        self.blink_consecutive_frames = 3
        self.blink_counter = 0
        self.is_blinking = False
        
        # Click states
        self.left_click_enabled = False
        self.right_click_enabled = False
        
    def get_eye_aspect_ratio(self, eye_landmarks):
        """Calculate Eye Aspect Ratio for blink detection"""
        # Vertical eye landmarks
        A = np.linalg.norm(np.array(eye_landmarks[1]) - np.array(eye_landmarks[5]))
        B = np.linalg.norm(np.array(eye_landmarks[2]) - np.array(eye_landmarks[4]))
        # Horizontal eye landmark
        C = np.linalg.norm(np.array(eye_landmarks[0]) - np.array(eye_landmarks[3]))
        
        ear = (A + B) / (2.0 * C)
        return ear
    
    def get_gaze_direction(self, landmarks, frame_shape):
        """Extract gaze direction from eye landmarks"""
        h, w = frame_shape[:2]
        
        # Get eye center points
        left_eye_center = np.mean([[landmarks[i].x * w, landmarks[i].y * h] for i in self.LEFT_EYE], axis=0)
        right_eye_center = np.mean([[landmarks[i].x * w, landmarks[i].y * h] for i in self.RIGHT_EYE], axis=0)
        
        # Average both eyes for more stable tracking
        gaze_point = (left_eye_center + right_eye_center) / 2
        
        return gaze_point
    
    def detect_blink(self, landmarks):
        """Detect eye blinks"""
        # Get eye landmarks
        left_eye_points = [[landmarks[i].x, landmarks[i].y] for i in self.LEFT_EYE]
        right_eye_points = [[landmarks[i].x, landmarks[i].y] for i in self.RIGHT_EYE]
        
        # Calculate EAR for both eyes
        left_ear = self.get_eye_aspect_ratio(left_eye_points[:6])  # Use first 6 points
        right_ear = self.get_eye_aspect_ratio(right_eye_points[:6])
        
        ear = (left_ear + right_ear) / 2.0
        
        if ear < self.blink_threshold:
            self.blink_counter += 1
        else:
            if self.blink_counter >= self.blink_consecutive_frames:
                return True
            self.blink_counter = 0
        
        return False

class VirtualKeyboard:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Virtual Eye-Controlled Keyboard")
        self.root.geometry("1200x400")
        self.root.configure(bg='black')
        
        # Keyboard layout
        self.keys_layout = [
            ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'Backspace'],
            ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', 'Delete'],
            ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'Enter'],
            ['Z', 'X', 'C', 'V', 'B', 'N', 'M', 'Space', 'Shift'],
            ['Ctrl', 'Alt', 'Tab', 'Esc', 'Home', 'End', 'Up', 'Down', 'Left', 'Right']
        ]
        
        self.buttons = {}
        self.current_highlighted = None
        self.text_output = tk.StringVar()
        
        self.setup_keyboard()
        
    def setup_keyboard(self):
        # Text output area
        output_frame = tk.Frame(self.root, bg='black')
        output_frame.pack(pady=10)
        
        tk.Label(output_frame, text="Output:", fg='white', bg='black', font=('Arial', 12)).pack(side=tk.LEFT)
        output_entry = tk.Entry(output_frame, textvariable=self.text_output, width=60, font=('Arial', 12))
        output_entry.pack(side=tk.LEFT, padx=10)
        
        # Keyboard frame
        keyboard_frame = tk.Frame(self.root, bg='black')
        keyboard_frame.pack(pady=20)
        
        for row_idx, row in enumerate(self.keys_layout):
            row_frame = tk.Frame(keyboard_frame, bg='black')
            row_frame.pack(pady=2)
            
            for col_idx, key in enumerate(row):
                width = 8
                if key in ['Space']:
                    width = 20
                elif key in ['Backspace', 'Delete', 'Enter', 'Shift']:
                    width = 12
                
                btn = tk.Button(
                    row_frame,
                    text=key,
                    width=width,
                    height=2,
                    font=('Arial', 10, 'bold'),
                    bg='gray30',
                    fg='white',
                    activebackground='blue',
                    command=lambda k=key: self.key_pressed(k)
                )
                btn.pack(side=tk.LEFT, padx=1, pady=1)
                self.buttons[key] = btn
    
    def key_pressed(self, key):
        """Handle key press events"""
        if key == 'Space':
            self.text_output.set(self.text_output.get() + ' ')
        elif key == 'Backspace':
            current = self.text_output.get()
            self.text_output.set(current[:-1])
        elif key == 'Delete':
            self.text_output.set("")
        elif key == 'Enter':
            self.text_output.set(self.text_output.get() + '\n')
        elif key in ['Ctrl', 'Alt', 'Tab', 'Esc', 'Home', 'End', 'Up', 'Down', 'Left', 'Right', 'Shift']:
            # Handle special keys with pyautogui
            pyautogui.press(key.lower())
        else:
            self.text_output.set(self.text_output.get() + key.lower())
    
    def highlight_key(self, key):
        """Highlight a key when eye is focused on it"""
        if self.current_highlighted:
            self.current_highlighted.configure(bg='gray30')
        
        if key in self.buttons:
            self.buttons[key].configure(bg='red')
            self.current_highlighted = self.buttons[key]
    
    def get_key_at_position(self, x, y):
        """Get which key is at the given screen position"""
        # Convert screen coordinates to keyboard coordinates
        # This is a simplified version - you'd need more precise mapping
        for key, button in self.buttons.items():
            try:
                btn_x = button.winfo_rootx()
                btn_y = button.winfo_rooty()
                btn_width = button.winfo_width()
                btn_height = button.winfo_height()
                
                if btn_x <= x <= btn_x + btn_width and btn_y <= y <= btn_y + btn_height:
                    return key
            except:
                continue
        return None

class EyeControlSystem:
    def __init__(self):
        self.eye_tracker = EyeTracker()
        self.virtual_keyboard = VirtualKeyboard()
        self.cap = None
        self.running = False
        
        # Control modes
        self.keyboard_mode = True
        self.mouse_mode = False
        
        # Dwell time for selections (in seconds)
        self.dwell_time = 2.0
        self.current_dwell_start = None
        self.current_target = None
        
    def start_camera(self):
        """Initialize camera"""
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open camera")
            return False
        return True
    
    def process_frame(self):
        """Process each frame for eye tracking"""
        if not self.cap:
            return None
            
        ret, frame = self.cap.read()
        if not ret:
            return None
        
        frame = cv2.flip(frame, 1)  # Mirror the frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.eye_tracker.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Get gaze direction
                gaze_point = self.eye_tracker.get_gaze_direction(face_landmarks.landmark, frame.shape)
                
                # Smooth the gaze point
                self.eye_tracker.eye_positions.append(gaze_point)
                smoothed_gaze = np.mean(self.eye_tracker.eye_positions, axis=0)
                
                # Map to screen coordinates
                screen_x = int(smoothed_gaze[0] * self.eye_tracker.screen_width / frame.shape[1])
                screen_y = int(smoothed_gaze[1] * self.eye_tracker.screen_height / frame.shape[0])
                
                # Detect blinks for clicking
                if self.eye_tracker.detect_blink(face_landmarks.landmark):
                    if self.keyboard_mode:
                        key = self.virtual_keyboard.get_key_at_position(screen_x, screen_y)
                        if key:
                            self.virtual_keyboard.key_pressed(key)
                    elif self.mouse_mode:
                        pyautogui.click(screen_x, screen_y)
                
                # Update interface
                if self.keyboard_mode:
                    key = self.virtual_keyboard.get_key_at_position(screen_x, screen_y)
                    if key:
                        self.virtual_keyboard.highlight_key(key)
                elif self.mouse_mode:
                    pyautogui.moveTo(screen_x, screen_y)
                
                # Draw eye tracking visualization
                cv2.circle(frame, (int(smoothed_gaze[0]), int(smoothed_gaze[1])), 5, (0, 255, 0), -1)
        
        return frame
    
    def run(self):
        """Main execution loop"""
        if not self.start_camera():
            return
        
        self.running = True
        
        # Start keyboard in separate thread
        keyboard_thread = threading.Thread(target=self.virtual_keyboard.root.mainloop)
        keyboard_thread.daemon = True
        keyboard_thread.start()
        
        while self.running:
            frame = self.process_frame()
            if frame is not None:
                # Add control instructions
                cv2.putText(frame, f"Mode: {'Keyboard' if self.keyboard_mode else 'Mouse'}", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, "Press 'k' for Keyboard, 'm' for Mouse, 'q' to quit", 
                           (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, "Blink to click/select", 
                           (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('Eye Tracking Control', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                self.running = False
                break
            elif key == ord('k'):
                self.keyboard_mode = True
                self.mouse_mode = False
            elif key == ord('m'):
                self.keyboard_mode = False
                self.mouse_mode = True
        
        self.cleanup()
    
    def cleanup(self):
        """Clean up resources"""
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()

# Initialize and run the system
if __name__ == "__main__":
    print("Eye-Tracking Keyboard and Mouse Control System")
    print("=" * 50)
    print("Features:")
    print("- Virtual keyboard control with eye tracking")
    print("- Mouse control with eye movement")
    print("- Blink detection for clicking/selecting")
    print("- Real-time webcam processing")
    print("\nInstructions:")
    print("1. Look at keys to highlight them")
    print("2. Blink to select/click")
    print("3. Press 'k' for keyboard mode, 'm' for mouse mode")
    print("4. Press 'q' to quit")
    print("\nStarting system...")
    
    system = EyeControlSystem()
    system.run()

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Enhanced Eye-Tracking System with Calibration and Advanced Features
import cv2
import numpy as np
import pyautogui
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import time
import math
from collections import deque
import json
import os

class AdvancedEyeTracker:
    def __init__(self):
        # Try to import mediapipe, fallback to basic tracking if not available
        self.use_mediapipe = False
        try:
            import mediapipe as mp
            self.mp_face_mesh = mp.solutions.face_mesh
            self.face_mesh = self.mp_face_mesh.FaceMesh(
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5
            )
            self.use_mediapipe = True
            print("✓ MediaPipe loaded successfully")
        except ImportError:
            print("⚠ MediaPipe not available, using basic face detection")
            # Initialize basic face/eye detection
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
        # Screen dimensions
        self.screen_width, self.screen_height = pyautogui.size()
        
        # Calibration data
        self.calibration_data = []
        self.is_calibrated = False
        
        # Smoothing and tracking
        self.gaze_history = deque(maxlen=10)
        self.blink_threshold = 0.3
        self.last_blink_time = 0
        self.blink_cooldown = 0.5
        
        # Settings
        self.sensitivity = 1.0
        self.dwell_time = 1.5
        
    def calibrate(self, screen_points, eye_points):
        """Calibrate eye tracking to screen coordinates"""
        if len(screen_points) >= 4 and len(eye_points) >= 4:
            # Use polynomial transformation for calibration
            screen_points = np.array(screen_points, dtype=np.float32)
            eye_points = np.array(eye_points, dtype=np.float32)
            
            # Calculate transformation matrix
            self.transform_matrix = cv2.getPerspectiveTransform(
                eye_points[:4], screen_points[:4]
            )
            self.is_calibrated = True
            return True
        return False
    
    def track_eyes_basic(self, frame):
        """Basic eye tracking using Haar cascades"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        gaze_point = None
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            eyes = self.eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                # Get center point between eyes
                eye_centers = []
                for (ex, ey, ew, eh) in eyes[:2]:
                    eye_center = (x + ex + ew//2, y + ey + eh//2)
                    eye_centers.append(eye_center)
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                
                if len(eye_centers) == 2:
                    gaze_point = np.mean(eye_centers, axis=0)
                    cv2.circle(frame, tuple(map(int, gaze_point)), 5, (255, 0, 0), -1)
        
        return gaze_point
    
    def track_eyes_mediapipe(self, frame):
        """Advanced eye tracking using MediaPipe"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        
        gaze_point = None
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w = frame.shape[:2]
                
                # Eye landmarks (simplified)
                left_eye_indices = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
                right_eye_indices = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
                
                # Calculate eye centers
                left_eye_points = [(int(face_landmarks.landmark[i].x * w), 
                                  int(face_landmarks.landmark[i].y * h)) for i in left_eye_indices]
                right_eye_points = [(int(face_landmarks.landmark[i].x * w), 
                                   int(face_landmarks.landmark[i].y * h)) for i in right_eye_indices]
                
                left_center = np.mean(left_eye_points, axis=0)
                right_center = np.mean(right_eye_points, axis=0)
                
                gaze_point = (left_center + right_center) / 2
                
                # Draw eye regions
                for point in left_eye_points + right_eye_points:
                    cv2.circle(frame, point, 1, (0, 255, 0), -1)
                
                cv2.circle(frame, tuple(map(int, gaze_point)), 5, (255, 0, 0), -1)
        
        return gaze_point
    
    def map_to_screen(self, gaze_point, frame_shape):
        """Map gaze point to screen coordinates"""
        if not self.is_calibrated:
            # Simple proportional mapping
            h, w = frame_shape[:2]
            screen_x = int(gaze_point[0] * self.screen_width / w)
            screen_y = int(gaze_point[1] * self.screen_height / h)
        else:
            # Use calibration matrix
            point = np.array([[gaze_point]], dtype=np.float32)
            transformed = cv2.perspectiveTransform(point, self.transform_matrix)
            screen_x, screen_y = transformed[0][0]
        
        # Apply sensitivity and smoothing
        self.gaze_history.append((screen_x, screen_y))
        if len(self.gaze_history) > 1:
            smoothed = np.mean(self.gaze_history, axis=0)
            return int(smoothed[0]), int(smoothed[1])
        
        return int(screen_x), int(screen_y)

class EnhancedVirtualKeyboard:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Advanced Eye-Controlled Keyboard")
        self.root.geometry("1400x500")
        self.root.configure(bg='#1a1a1a')
        
        # Keyboard layouts
        self.qwerty_layout = [
            ['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', 'Backspace'],
            ['Tab', 'Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '[', ']', '\\'],
            ['Caps', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ';', "'", 'Enter'],
            ['Shift', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', ',', '.', '/', 'Shift'],
            ['Ctrl', 'Win', 'Alt', 'Space', 'Alt', 'Menu', 'Ctrl', '←', '↑', '↓', '→']
        ]
        
        self.buttons = {}
        self.current_highlighted = None
        self.text_output = tk.StringVar()
        self.caps_lock = False
        self.shift_pressed = False
        
        # Dwell selection
        self.dwell_start_time = None
        self.dwell_key = None
        self.dwell_duration = 2.0
        
        self.setup_ui()
        
    def setup_ui(self):
        # Control panel
        control_frame = tk.Frame(self.root, bg='#1a1a1a')
        control_frame.pack(pady=10)
        
        tk.Label(control_frame, text="Eye-Controlled Keyboard", 
                fg='white', bg='#1a1a1a', font=('Arial', 16, 'bold')).pack()
        
        # Text output
        output_frame = tk.Frame(self.root, bg='#1a1a1a')
        output_frame.pack(pady=10)
        
        tk.Label(output_frame, text="Output:", fg='white', bg='#1a1a1a', 
                font=('Arial', 12)).pack(anchor='w')
        
        self.text_area = tk.Text(output_frame, width=80, height=4, font=('Arial', 12))
        self.text_area.pack(pady=5)
        
        # Keyboard
        keyboard_frame = tk.Frame(self.root, bg='#1a1a1a')
        keyboard_frame.pack(pady=20)
        
        for row_idx, row in enumerate(self.qwerty_layout):
            row_frame = tk.Frame(keyboard_frame, bg='#1a1a1a')
            row_frame.pack(pady=2)
            
            for key in row:
                width = self.get_key_width(key)
                
                btn = tk.Button(
                    row_frame,
                    text=key,
                    width=width,
                    height=2,
                    font=('Arial', 10, 'bold'),
                    bg='#404040',
                    fg='white',
                    activebackground='#0066cc',
                    relief='raised',
                    command=lambda k=key: self.key_pressed(k)
                )
                btn.pack(side=tk.LEFT, padx=1, pady=1)
                self.buttons[key] = btn
        
        # Status bar
        status_frame = tk.Frame(self.root, bg='#1a1a1a')
        status_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=5)
        
        self.status_label = tk.Label(status_frame, text="Ready", 
                                   fg='white', bg='#1a1a1a', font=('Arial', 10))
        self.status_label.pack(side=tk.LEFT, padx=10)
        
        # Dwell progress bar
        self.dwell_progress = ttk.Progressbar(status_frame, length=200, mode='determinate')
        self.dwell_progress.pack(side=tk.RIGHT, padx=10)
    
    def get_key_width(self, key):
        """Get appropriate width for different keys"""
        if key == 'Space':
            return 25
        elif key in ['Backspace', 'Enter', 'Shift', 'Tab']:
            return 12
        elif key in ['Caps', 'Ctrl', 'Alt', 'Win']:
            return 8
        else:
            return 4
    
    def key_pressed(self, key):
        """Handle key press events"""
        current_text = self.text_area.get('1.0', tk.END)
        
        if key == 'Space':
            self.text_area.insert(tk.END, ' ')
        elif key == 'Backspace':
            self.text_area.delete('end-2c', 'end-1c')
        elif key == 'Enter':
            self.text_area.insert(tk.END, '\n')
        elif key == 'Tab':
            self.text_area.insert(tk.END, '\t')
        elif key == 'Caps':
            self.caps_lock = not self.caps_lock
            self.update_caps_indicator()
        elif key == 'Shift':
            self.shift_pressed = not self.shift_pressed
            self.update_shift_indicator()
        elif key in ['Ctrl', 'Alt', 'Win', 'Menu']:
            # Handle modifier keys
            pass
        elif key in ['←', '↑', '↓', '→']:
            # Handle arrow keys
            if key == '←':
                self.text_area.mark_set(tk.INSERT, 'insert-1c')
            elif key == '→':
                self.text_area.mark_set(tk.INSERT, 'insert+1c')
        else:
            # Regular character
            char = key
            if self.caps_lock or self.shift_pressed:
                char = char.upper()
            else:
                char = char.lower()
            
            self.text_area.insert(tk.END, char)
            
            if self.shift_pressed:
                self.shift_pressed = False
                self.update_shift_indicator()
        
        self.text_area.see(tk.END)
    
    def update_caps_indicator(self):
        """Update caps lock indicator"""
        if self.caps_lock:
            self.buttons['Caps'].configure(bg='#cc6600')
        else:
            self.buttons['Caps'].configure(bg='#404040')
    
    def update_shift_indicator(self):
        """Update shift indicator"""
        color = '#cc6600' if self.shift_pressed else '#404040'
        for key in ['Shift']:
            if key in self.buttons:
                self.buttons[key].configure(bg=color)
    
    def highlight_key(self, key, progress=0):
        """Highlight key with dwell progress"""
        # Reset previous highlight
        if self.current_highlighted and self.current_highlighted != key:
            self.buttons[self.current_highlighted].configure(bg='#404040')
        
        if key in self.buttons:
            # Color intensity based on dwell progress
            intensity = min(255, int(100 + progress * 155))
            color = f'#{intensity:02x}4040'
            self.buttons[key].configure(bg=color)
            self.current_highlighted = key
            
            # Update progress bar
            self.dwell_progress['value'] = progress * 100
    
    def get_key_at_position(self, x, y):
        """Get key at screen position"""
        self.root.update()  # Ensure geometry is current
        
        for key, button in self.buttons.items():
            try:
                btn_x = button.winfo_rootx()
                btn_y = button.winfo_rooty()
                btn_width = button.winfo_width()
                btn_height = button.winfo_height()
                
                if (btn_x <= x <= btn_x + btn_width and 
                    btn_y <= y <= btn_y + btn_height):
                    return key
            except tk.TclError:
                continue
        return None
    
    def update_status(self, message):
        """Update status message"""
        self.status_label.configure(text=message)

class EyeControlSystem:
    def __init__(self):
        self.eye_tracker = AdvancedEyeTracker()
        self.virtual_keyboard = EnhancedVirtualKeyboard()
        self.cap = None
        self.running = False
        
        # Control modes
        self.keyboard_mode = True
        self.mouse_mode = False
        self.calibration_mode = False
        
        # Dwell selection
        self.dwell_start_time = None
        self.current_target = None
        self.dwell_duration = 2.0
        
        # Calibration
        self.calibration_points = []
        self.calibration_eye_points = []
        
    def start_camera(self):
        """Initialize camera with error handling"""
        for i in range(3):  # Try multiple camera indices
            self.cap = cv2.VideoCapture(i)
            if self.cap.isOpened():
                # Set camera properties for better performance
                self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
                self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
                self.cap.set(cv2.CAP_PROP_FPS, 30)
                print(f"✓ Camera {i} initialized successfully")
                return True
        
        print("✗ Could not initialize any camera")
        messagebox.showerror("Error", "Could not access camera")
        return False
    
    def calibrate_system(self):
        """Interactive calibration process"""
        if not self.cap:
            return
        
        calibration_points = [
            (100, 100), (self.eye_tracker.screen_width - 100, 100),
            (100, self.eye_tracker.screen_height - 100), 
            (self.eye_tracker.screen_width - 100, self.eye_tracker.screen_height - 100),
            (self.eye_tracker.screen_width // 2, self.eye_tracker.screen_height // 2)
        ]
        
        print("Starting calibration...")
        self.virtual_keyboard.update_status("Calibration mode - Look at the red circles")
        
        eye_points = []
        
        for i, (cx, cy) in enumerate(calibration_points):
            print(f"Look at point {i+1}/5")
            
            # Create calibration window
            cal_window = np.zeros((self.eye_tracker.screen_height, 
                                 self.eye_tracker.screen_width, 3), dtype=np.uint8)
            
            start_time = time.time()
            collected_points = []
            
            while time.time() - start_time < 3.0:  # 3 seconds per point
                ret, frame = self.cap.read()
                if not ret:
                    continue
                
                frame = cv2.flip(frame, 1)
                
                # Track eyes
                if self.eye_tracker.use_mediapipe:
                    gaze_point = self.eye_tracker.track_eyes_mediapipe(frame)
                else:
                    gaze_point = self.eye_tracker.track_eyes_basic(frame)
                
                if gaze_point is not None:
                    collected_points.append(gaze_point)
                
                # Draw calibration point
                cv2.circle(cal_window, (cx, cy), 20, (0, 0, 255), -1)
                cv2.circle(cal_window, (cx, cy), 40, (0, 0, 255), 2)
                
                # Show progress
                progress = (time.time() - start_time) / 3.0
                cv2.putText(cal_window, f"Point {i+1}/5 - {progress*100:.0f}%", 
                           (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                
                cv2.imshow('Calibration', cal_window)
                cv2.imshow('Eye Tracking', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            if collected_points:
                avg_point = np.mean(collected_points, axis=0)
                eye_points.append(avg_point)
                print(f"Collected point {i+1}: {avg_point}")
        
        cv2.destroyWindow('Calibration')
        
        # Perform calibration
        if len(eye_points) >= 4:
            success = self.eye_tracker.calibrate(calibration_points, eye_points)
            if success:
                print("✓ Calibration successful!")
                self.virtual_keyboard.update_status("Calibrated - Ready for eye control")
                return True
            else:
                print("✗ Calibration failed")
                self.virtual_keyboard.update_status("Calibration failed")
        
        return False
    
    def process_frame(self):
        """Process each frame for eye tracking"""
        if not self.cap:
            return None
        
        ret, frame = self.cap.read()
        if not ret:
            return None
        
        frame = cv2.flip(frame, 1)
        
        # Track eyes
        if self.eye_tracker.use_mediapipe:
            gaze_point = self.eye_tracker.track_eyes_mediapipe(frame)
        else:
            gaze_point = self.eye_tracker.track_eyes_basic(frame)
        
        if gaze_point is not None:
            # Map to screen coordinates
            screen_x, screen_y = self.eye_tracker.map_to_screen(gaze_point, frame.shape)
            
            # Handle different modes
            if self.keyboard_mode:
                self.handle_keyboard_mode(screen_x, screen_y)
            elif self.mouse_mode:
                self.handle_mouse_mode(screen_x, screen_y)
        
        return frame
    
    def handle_keyboard_mode(self, screen_x, screen_y):
        """Handle keyboard interaction"""
        key = self.virtual_keyboard.get_key_at_position(screen_x, screen_y)
        
        if key:
            current_time = time.time()
            
            if key != self.current_target:
                # New target
                self.current_target = key
                self.dwell_start_time = current_time
                
            else:
                # Same target - check dwell time
                dwell_elapsed = current_time - self.dwell_start_time
                progress = min(1.0, dwell_elapsed / self.dwell_duration)
                
                self.virtual_keyboard.highlight_key(key, progress)
                
                if dwell_elapsed >= self.dwell_duration:
                    # Execute key press
                    self.virtual_keyboard.key_pressed(key)
                    self.current_target = None
                    self.dwell_start_time = None
                    print(f"Key pressed: {key}")
        else:
            # No target
            if self.current_target:
                self.virtual_keyboard.highlight_key(self.current_target, 0)
            self.current_target = None
            self.dwell_start_time = None
    
    def handle_mouse_mode(self, screen_x, screen_y):
        """Handle mouse control"""
        # Smooth mouse movement
        try:
            pyautogui.moveTo(screen_x, screen_y, duration=0.01)
        except:
            pass
    
    def run(self):
        """Main execution loop"""
        if not self.start_camera():
            return
        
        # Start keyboard interface
        keyboard_thread = threading.Thread(target=self.virtual_keyboard.root.mainloop)
        keyboard_thread.daemon = True
        keyboard_thread.start()
        
        print("System ready!")
        print("Controls:")
        print("- 'c' to calibrate")
        print("- 'k' for keyboard mode")
        print("- 'm' for mouse mode")
        print("- 'q' to quit")
        
        self.running = True
        
        while self.running:
            frame = self.process_frame()
            
            if frame is not None:
                # Add UI overlay
                self.add_overlay(frame)
                cv2.imshow('Eye Control System', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                self.running = False
                break
            elif key == ord('c'):
                self.calibrate_system()
            elif key == ord('k'):
                self.keyboard_mode = True
                self.mouse_mode = False
                self.virtual_keyboard.update_status("Keyboard mode active")
            elif key == ord('m'):
                self.keyboard_mode = False
                self.mouse_mode = True
                self.virtual_keyboard.update_status("Mouse mode active")
        
        self.cleanup()
    
    def add_overlay(self, frame):
        """Add information overlay to frame"""
        h, w = frame.shape[:2]
        
        # Status text
        mode_text = "Keyboard" if self.keyboard_mode else "Mouse"
        cv2.putText(frame, f"Mode: {mode_text}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        calib_status = "Calibrated" if self.eye_tracker.is_calibrated else "Not Calibrated"
        cv2.putText(frame, f"Calibration: {calib_status}", (10, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        # Control instructions
        cv2.putText(frame, "Controls: 'c'=Calibrate 'k'=Keyboard 'm'=Mouse 'q'=Quit", 
                   (10, h-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Dwell indicator
        if self.current_target and self.dwell_start_time:
            elapsed = time.time() - self.dwell_start_time
            progress = min(1.0, elapsed / self.dwell_duration)
            
            # Progress bar
            bar_width = 200
            bar_height = 20
            bar_x = w - bar_width - 10
            bar_y = 10
            
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), 
                         (50, 50, 50), -1)
            cv2.rectangle(frame, (bar_x, bar_y), 
                         (bar_x + int(bar_width * progress), bar_y + bar_height), 
                         (0, 255, 0), -1)
            cv2.putText(frame, f"Target: {self.current_target}", 
                       (bar_x, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    def cleanup(self):
        """Clean up resources"""
        self.running = False
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        try:
            self.virtual_keyboard.root.quit()
        except:
            pass

# Demo function to test the system
def run_eye_control_demo():
    """Run the eye control system demo"""
    print("🔥 Advanced Eye-Controlled Keyboard & Mouse System")
    print("=" * 60)
    print("Features:")
    print("✓ Advanced eye tracking with MediaPipe fallback")
    print("✓ Full QWERTY virtual keyboard")
    print("✓ Mouse control with eye movement")
    print("✓ Calibration system for accuracy")
    print("✓ Dwell-time selection (no clicking needed)")
    print("✓ Real-time visual feedback")
    print("✓ Smooth eye movement tracking")
    print("")
    print("Getting ready...")
    
    try:
        system = EyeControlSystem()
        system.run()
    except KeyboardInterrupt:
        print("\nSystem stopped by user")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cv2.destroyAllWindows()

# Run the demo
if __name__ == "__main__":
    run_eye_control_demo()

In [ ]:
# Simplified Eye-Tracking System (OpenCV Only) - Ready to Test
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
import threading
import time
import math

# Simple eye tracking using OpenCV only
class SimpleEyeTracker:
    def __init__(self):
        # Load cascade classifiers
        try:
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
            print("✓ OpenCV cascades loaded successfully")
        except Exception as e:
            print(f"Error loading cascades: {e}")
            return
        
        # Mouse control (if pyautogui is available)
        self.mouse_control = False
        try:
            import pyautogui
            self.pyautogui = pyautogui
            self.mouse_control = True
            self.screen_width, self.screen_height = pyautogui.size()
            pyautogui.FAILSAFE = False
            print("✓ Mouse control enabled")
        except ImportError:
            print("⚠ PyAutoGUI not available - mouse control disabled")
        
        # Tracking variables
        self.last_gaze_point = None
        self.gaze_history = []
        self.max_history = 5
        
    def detect_eyes_and_face(self, frame):
        """Detect face and eyes in frame"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
        
        results = {'faces': [], 'eyes': [], 'gaze_point': None}
        
        for (x, y, w, h) in faces:
            results['faces'].append((x, y, w, h))
            
            # Region of interest for eyes
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            # Detect eyes in face region
            eyes = self.eye_cascade.detectMultiScale(roi_gray, 1.1, 3)
            
            eye_centers = []
            for (ex, ey, ew, eh) in eyes:
                # Adjust coordinates to full frame
                eye_x = x + ex + ew // 2
                eye_y = y + ey + eh // 2
                eye_centers.append((eye_x, eye_y))
                results['eyes'].append((x + ex, y + ey, ew, eh))
            
            # Calculate gaze point (center between eyes)
            if len(eye_centers) >= 2:
                gaze_x = sum(center[0] for center in eye_centers[:2]) // 2
                gaze_y = sum(center[1] for center in eye_centers[:2]) // 2
                results['gaze_point'] = (gaze_x, gaze_y)
        
        return results
    
    def smooth_gaze(self, gaze_point):
        """Apply smoothing to gaze point"""
        if gaze_point is None:
            return self.last_gaze_point
        
        self.gaze_history.append(gaze_point)
        if len(self.gaze_history) > self.max_history:
            self.gaze_history.pop(0)
        
        if len(self.gaze_history) > 0:
            avg_x = sum(p[0] for p in self.gaze_history) / len(self.gaze_history)
            avg_y = sum(p[1] for p in self.gaze_history) / len(self.gaze_history)
            smoothed = (int(avg_x), int(avg_y))
            self.last_gaze_point = smoothed
            return smoothed
        
        return gaze_point

class SimpleVirtualKeyboard:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Simple Eye-Controlled Keyboard")
        self.root.geometry("900x300")
        self.root.configure(bg='#2d2d2d')
        
        # Simple keyboard layout
        self.layout = [
            ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'Del'],
            ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', 'Clear'],
            ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'Enter'],
            ['Z', 'X', 'C', 'V', 'B', 'N', 'M', 'Space']
        ]
        
        self.buttons = {}
        self.highlighted_key = None
        self.setup_keyboard()
        
        # Output text
        self.output_text = ""
        
    def setup_keyboard(self):
        # Title
        title_label = tk.Label(self.root, text="Eye-Controlled Virtual Keyboard", 
                              fg='white', bg='#2d2d2d', font=('Arial', 14, 'bold'))
        title_label.pack(pady=5)
        
        # Output display
        self.output_var = tk.StringVar()
        output_frame = tk.Frame(self.root, bg='#2d2d2d')
        output_frame.pack(pady=5)
        
        tk.Label(output_frame, text="Output:", fg='white', bg='#2d2d2d').pack(side=tk.LEFT)
        self.output_entry = tk.Entry(output_frame, textvariable=self.output_var, 
                                   width=50, font=('Arial', 12))
        self.output_entry.pack(side=tk.LEFT, padx=10)
        
        # Keyboard
        keyboard_frame = tk.Frame(self.root, bg='#2d2d2d')
        keyboard_frame.pack(pady=10)
        
        for row in self.layout:
            row_frame = tk.Frame(keyboard_frame, bg='#2d2d2d')
            row_frame.pack(pady=2)
            
            for key in row:
                width = 8 if key != 'Space' else 20
                
                btn = tk.Button(row_frame, text=key, width=width, height=2,
                              font=('Arial', 10, 'bold'), bg='#555', fg='white',
                              command=lambda k=key: self.key_pressed(k))
                btn.pack(side=tk.LEFT, padx=2)
                self.buttons[key] = btn
        
        # Status label
        self.status_label = tk.Label(self.root, text="Ready - Look at keys to highlight", 
                                   fg='yellow', bg='#2d2d2d', font=('Arial', 10))
        self.status_label.pack(pady=5)
    
    def key_pressed(self, key):
        """Handle virtual key presses"""
        if key == 'Space':
            self.output_text += ' '
        elif key == 'Del':
            self.output_text = self.output_text[:-1]
        elif key == 'Clear':
            self.output_text = ""
        elif key == 'Enter':
            self.output_text += '\n'
        else:
            self.output_text += key.lower()
        
        self.output_var.set(self.output_text)
        print(f"Key pressed: {key}")
    
    def highlight_key(self, key):
        """Highlight a key"""
        # Reset previous highlight
        if self.highlighted_key and self.highlighted_key in self.buttons:
            self.buttons[self.highlighted_key].configure(bg='#555')
        
        # Highlight new key
        if key in self.buttons:
            self.buttons[key].configure(bg='#ff6600')
            self.highlighted_key = key
            self.status_label.configure(text=f"Focusing on: {key}")
    
    def get_key_at_position(self, x, y):
        """Get key at screen coordinates"""
        self.root.update()
        
        for key, button in self.buttons.items():
            try:
                btn_x = button.winfo_rootx()
                btn_y = button.winfo_rooty()
                btn_width = button.winfo_width()
                btn_height = button.winfo_height()
                
                if (btn_x <= x <= btn_x + btn_width and 
                    btn_y <= y <= btn_y + btn_height):
                    return key
            except:
                continue
        return None

class SimpleEyeControlSystem:
    def __init__(self):
        self.tracker = SimpleEyeTracker()
        self.keyboard = SimpleVirtualKeyboard()
        self.cap = None
        self.running = False
        
        # Selection timing
        self.focus_start_time = None
        self.current_focus_key = None
        self.selection_delay = 2.0  # 2 seconds to select
        
        # Modes
        self.keyboard_mode = True
        self.mouse_mode = False
        
    def start_camera(self):
        """Start camera capture"""
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("❌ Could not open camera")
            return False
        
        print("✅ Camera started successfully")
        return True
    
    def process_frame(self):
        """Process camera frame"""
        if not self.cap:
            return None
        
        ret, frame = self.cap.read()
        if not ret:
            return None
        
        frame = cv2.flip(frame, 1)  # Mirror image
        
        # Detect eyes and face
        detection_results = self.tracker.detect_eyes_and_face(frame)
        
        # Draw detections
        self.draw_detections(frame, detection_results)
        
        # Handle gaze tracking
        if detection_results['gaze_point']:
            gaze_point = self.tracker.smooth_gaze(detection_results['gaze_point'])
            if gaze_point:
                self.handle_gaze(gaze_point, frame.shape)
        
        return frame
    
    def draw_detections(self, frame, results):
        """Draw detection results on frame"""
        # Draw face rectangles
        for (x, y, w, h) in results['faces']:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Draw eye rectangles
        for (x, y, w, h) in results['eyes']:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw gaze point
        if results['gaze_point']:
            gx, gy = results['gaze_point']
            cv2.circle(frame, (gx, gy), 10, (0, 0, 255), -1)
            cv2.circle(frame, (gx, gy), 20, (0, 0, 255), 2)
    
    def handle_gaze(self, gaze_point, frame_shape):
        """Handle gaze point for interaction"""
        if self.keyboard_mode:
            self.handle_keyboard_gaze(gaze_point)
        elif self.mouse_mode and self.tracker.mouse_control:
            self.handle_mouse_gaze(gaze_point, frame_shape)
    
    def handle_keyboard_gaze(self, gaze_point):
        """Handle gaze for keyboard interaction"""
        # Map gaze to screen coordinates (approximate)
        screen_x = int(gaze_point[0] * 2)  # Simple scaling
        screen_y = int(gaze_point[1] * 2)
        
        # Get key at position
        key = self.keyboard.get_key_at_position(screen_x, screen_y)
        
        if key:
            current_time = time.time()
            
            if key != self.current_focus_key:
                # New key focused
                self.current_focus_key = key
                self.focus_start_time = current_time
                self.keyboard.highlight_key(key)
            else:
                # Same key - check selection time
                if current_time - self.focus_start_time >= self.selection_delay:
                    # Select the key
                    self.keyboard.key_pressed(key)
                    self.current_focus_key = None
                    self.focus_start_time = None
        else:
            # No key focused
            self.current_focus_key = None
            self.focus_start_time = None
    
    def handle_mouse_gaze(self, gaze_point, frame_shape):
        """Handle gaze for mouse control"""
        h, w = frame_shape[:2]
        # Map to screen coordinates
        screen_x = int(gaze_point[0] * self.tracker.screen_width / w)
        screen_y = int(gaze_point[1] * self.tracker.screen_height / h)
        
        try:
            self.tracker.pyautogui.moveTo(screen_x, screen_y, duration=0.1)
        except:
            pass
    
    def add_overlay_info(self, frame):
        """Add information overlay to frame"""
        h, w = frame.shape[:2]
        
        # Mode indicator
        mode_text = "KEYBOARD" if self.keyboard_mode else "MOUSE"
        cv2.putText(frame, f"Mode: {mode_text}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Instructions
        cv2.putText(frame, "Controls: K=Keyboard, M=Mouse, Q=Quit", 
                   (10, h - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Selection timer
        if self.current_focus_key and self.focus_start_time:
            elapsed = time.time() - self.focus_start_time
            progress = min(1.0, elapsed / self.selection_delay)
            
            # Progress bar
            bar_width = 200
            bar_height = 20
            bar_x = w - bar_width - 10
            bar_y = 50
            
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), 
                         (50, 50, 50), -1)
            cv2.rectangle(frame, (bar_x, bar_y), 
                         (bar_x + int(bar_width * progress), bar_y + bar_height), 
                         (0, 255, 0), -1)
            cv2.putText(frame, f"Selecting: {self.current_focus_key}", 
                       (bar_x, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    def run(self):
        """Main run loop"""
        if not self.start_camera():
            return
        
        # Start keyboard in separate thread
        keyboard_thread = threading.Thread(target=self.keyboard.root.mainloop)
        keyboard_thread.daemon = True
        keyboard_thread.start()
        
        print("\n🚀 Simple Eye Control System Started!")
        print("👁️  Look at keyboard keys to highlight them")
        print("⏱️  Hold gaze for 2 seconds to select")
        print("⌨️  Press 'K' for keyboard mode")
        print("🖱️  Press 'M' for mouse mode")
        print("❌ Press 'Q' to quit")
        
        self.running = True
        
        while self.running:
            frame = self.process_frame()
            if frame is not None:
                self.add_overlay_info(frame)
                cv2.imshow('Simple Eye Control', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                self.running = False
                break
            elif key == ord('k'):
                self.keyboard_mode = True
                self.mouse_mode = False
                print("🔄 Switched to Keyboard mode")
            elif key == ord('m'):
                self.keyboard_mode = False
                self.mouse_mode = True
                print("🔄 Switched to Mouse mode")
        
        self.cleanup()
    
    def cleanup(self):
        """Clean up resources"""
        print("\n🛑 Shutting down...")
        self.running = False
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        try:
            self.keyboard.root.quit()
        except:
            pass
        print("✅ Cleanup completed")

# Quick test function
def test_simple_eye_control():
    """Test the simple eye control system"""
    print("🎯 Simple Eye Control System - OpenCV Version")
    print("=" * 50)
    print("This version uses only OpenCV and works immediately!")
    print("")
    
    try:
        system = SimpleEyeControlSystem()
        system.run()
    except KeyboardInterrupt:
        print("\n⚠️ System stopped by user")
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

# Run the simple version
test_simple_eye_control()